# Mood prediction with TensorFlow
Trying to predict the mood of a user each day with:
- Mood rated between 0 and 5
- Weather rated between 0 and 5
- Temperature
- Month of the year
- Day of the week between 0 and 6

In [1]:
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input

In [2]:
mood_history = pd.read_csv('mood-history.csv').sort_values('timestamp')
mood_history.head()

,timestamp,mood,weather,temperature,month,day,weekday
0,1616320024,5,5,8,2,21,6
1,1616406424,3,4,8,2,22,0
2,1616579224,4,5,9,2,23,1
3,1616579224,3,5,9,2,24,2
4,1616665624,2,4,7,2,25,3


## Data normalization

Normalizes some input features to make them all between 0 and 1.

Applies `(x - min) / (max - min)` for all the features.

In [3]:
mood_history['mood'] = mood_history['mood'].apply(lambda x: x / 5)
mood_history['weather'] = mood_history['weather'].apply(lambda x: x / 5)
mood_history['temperature'] = mood_history['temperature'].apply(lambda x: (x + 30) / (50 + 30))

def init_bag(size, index):
    """
    Initialize a bag of a specified size with an index in which the value
    will be 1.
    """

    bag = [0] * size
    bag[index] = 1

    return bag

mood_history['month'] = mood_history['month'].apply(lambda x: init_bag(12, x))
mood_history['day'] = mood_history['day'].apply(lambda x: init_bag(31, x))
mood_history['weekday'] = mood_history['weekday'].apply(lambda x: init_bag(7, x))

mood_history

,timestamp,mood,weather,temperature,month,day,weekday
0,1616320024,1.0,1.0,0.4750,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1]"
1,1616406424,0.6,0.8,0.4750,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0]"
2,1616579224,0.8,1.0,0.4875,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0]"
3,1616579224,0.6,1.0,0.4875,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0]"
4,1616665624,0.4,0.8,0.4625,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0]"
5,1616752024,0.4,0.6,0.4375,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0]"
6,1616838424,1.0,1.0,0.5250,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0]"


In [4]:
X_train = mood_history[[
            'weather',
            'temperature',
            'month',
            'day',
            'weekday'
]]
# Create new columns as bags for months, days and weekday
X_train[[f'm{i}' for i in range(0, 12)]] = X_train['month'].apply(pd.Series)
X_train[[f'd{i}' for i in range(0, 31)]] = X_train['day'].apply(pd.Series)
X_train[[f'w{i}' for i in range(0, 7)]] = X_train['weekday'].apply(pd.Series)

# Drop the old columns
X_train.drop(['month', 'day', 'weekday'], inplace=True, axis=1)

# Remove columns names for integers
X_train.columns = range(X_train.shape[1])

X_train.head()

/opt/anaconda3/envs/datalore-user/lib/python3.7/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,1.0,0.4750,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0.8,0.4750,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,1.0,0.4875,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1.0,0.4875,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0.8,0.4625,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [5]:
X_train.shape

(7, 52)

In [6]:
Y_train = mood_history[[
            'mood'
]]
Y_train.columns = range(Y_train.shape[1])

Y_train.head()

,0
0,1.0
1,0.6
2,0.8
3,0.6
4,0.4


In [7]:
model = Sequential()

model.add(Input(shape=(X_train.shape[1],)))
model.add(Dense(25, activation='sigmoid'))
model.add(Dense(15, activation='sigmoid'))
model.add(Dense(1))

In [8]:
model.compile(optimizer='sgd', loss='mean_squared_error')

In [9]:
model.fit(X_train, Y_train, batch_size=1, epochs=10)

Epoch 1/10
7/7 [==============================] - 0s 1ms/step - loss: 0.3968
Epoch 2/10
7/7 [==============================] - 0s 1ms/step - loss: 0.1217
Epoch 3/10
7/7 [==============================] - 0s 2ms/step - loss: 0.0769
Epoch 4/10
7/7 [==============================] - 0s 2ms/step - loss: 0.0693
Epoch 5/10
7/7 [==============================] - 0s 1ms/step - loss: 0.0680
Epoch 6/10
7/7 [==============================] - 0s 1ms/step - loss: 0.0669
Epoch 7/10
7/7 [==============================] - 0s 1ms/step - loss: 0.0675
Epoch 8/10
7/7 [==============================] - 0s 1ms/step - loss: 0.0675
Epoch 9/10
7/7 [==============================] - 0s 1ms/step - loss: 0.0671
Epoch 10/10
7/7 [==============================] - 0s 1ms/step - loss: 0.0663


In [10]:
predictions = model.predict(pd.DataFrame([[
            1,
            1,
            0,
                
]]))

predictions

ValueError: ValueError: in user code:

    /opt/anaconda3/envs/datalore-user/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1147 predict_function  *
        outputs = self.distribute_strategy.run(
    /opt/anaconda3/envs/datalore-user/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/anaconda3/envs/datalore-user/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/anaconda3/envs/datalore-user/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/anaconda3/envs/datalore-user/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1122 predict_step  **
        return self(x, training=False)
    /opt/anaconda3/envs/d